In [22]:
import pandas as pd
import numpy as np
import json
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import sys

import pickle


%matplotlib inline
rand_seed = 0  # random state for reproducibility
np.random.seed(rand_seed)


In [40]:
Data = pd.read_excel('ALL_data.xlsx')
Data = data.dropna()
Data.head()


,comment,label,clean
0,نفسي يوم تكتبو السعر بدون مانسال,سلبي,نفسي يوم تكتبو السعر بدون مانسال
2,لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...,سلبي,لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
4,واو,ايجابي,واو
8,شكرا التوضيح مفيد اكرر الشكر سوداني الابداع وا...,ايجابي,شكرا التوضيح مفيد اكرر الشكر سوداني الابداع وا...
13,سوداني جميل,ايجابي,سوداني جميل


In [44]:
data = Data[Data['label'] != 'محايد']
data['clean']=data['comment']


In [25]:
import stanza


nlp = stanza.Pipeline(lang='ar', processors='tokenize,lemma')

2023-01-02 21:43:41 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-02 21:43:43 WARNING: Language ar package default expects mwt, which has been added
2023-01-02 21:43:43 INFO: Loading these models for language: ar (Arabic):
| Processor | Package |
-----------------------
| tokenize  | padt    |
| mwt       | padt    |
| lemma     | padt    |

2023-01-02 21:43:43 INFO: Use device: cpu
2023-01-02 21:43:43 INFO: Loading: tokenize
2023-01-02 21:43:43 INFO: Loading: mwt
2023-01-02 21:43:43 INFO: Loading: lemma
2023-01-02 21:43:43 INFO: Done loading processors!


In [45]:
print(data['clean'][1])

KeyError: 1

In [30]:
for i in data['clean']:
    print(i)
   

نفسي يوم تكتبو السعر بدون مانسال
 لو قللتو الرسائل دي واديتونا ليها ميقات يكون اجمل
 واو
شكرا التوضيح مفيد اكرر الشكر سوداني الابداع والتميز
سوداني جميل
خدماتكم كلها ضابطه بس انا زعلانه من حاجه وحدة بس ( لي شلتو مني شارة أبرز المعجبين )
طبعا خجلانين تكتبوا السعر بعد الزيادات الأخيرة صح
الشبكه تعبانه شديد حاولوا حلو المشكله عشان م نشترك واشتراكنا يروح علي الفاضي ساكت تحياتي ليكم من منطقة ابو فسيسية حلة الدرحمان ابو طباعة الفجغ الجماعة
 بتبالق لكن نشطة قيقا و نزلت ملف و م ديرها تنتهي كمان
الشبكة عندنا كعبه جدا نتمني عن يتحسن ان شاءالله
ما عندنا شبكة لينا شهر وزيادة
سوداني تعبانة والله حتي ولو الخدمه مجان
صلحوا الشبكة اول بعد فالحين في الدفع بس
يوم العيد عشمتموني قلتو لي منوزع هدايا باقات نت مجانية عيدية وما اديتني حاجة منها
والله اول مره يكون كلامكم مشيت اشتريت شريحه جديده وفعلا لقيت فيها غيغا ونص هديه
لسوء الخدمة وقفت الحساب لاني بدفع وما مستفيد من خدمة الخط اتفاجئ بإن الشريحة بقيت غير خط لماذا؟
اناسوداني هيي هيي انااصل الحضارة
وقفو اي باقه وصلحو الشبكه
نستمتع كيف وانتو رفعتوها السماء"


عايزين يصلو وين قال
وحات الله حرام عليك
امس اتصدمت عديل من ارتفاع اسعاركم
هسي بس بقيت جاري واي فاي بس لبتين لبتين
ارفعو ساي اصلا م عندنا شغله بسوداني
التبش زاد ياهو ده السودان
تسقط بس
ماشين اريبا بس
وبعد ده كلو شبكتكم كعبه شديد
حرام عليكم اه اقلب اربيا
اكترر من كدهه ديل رفعو ضغطنااا
بنعمل خدمه هسي لمده شهر وتاني بعد تنتهي بكشح الشريحه
نرجع ونعمل اضراب اكبرواشمل مقاطعت شركات الاتصالات السودانيه
الله اكبر عليكم
النت خليناه
احلي حاجه ال و ضع المجاني بس
تسقط بس
والله والله كرهتونا سواني زتو
احسن تقاطعوا الشركات دي عشان تخسر وتعرف الله واحد
بنخلي االنت ذاتوو
نور نور العين ديل الامس شكرناهم ومدحناهم
قلت النبز في سري
التواصل بالارواح
بنخلي الانترنت
بعد كدا نرجع للحمام الزاجل بس
مكاله الا رد زاتو
عشان قلت دايره اخلي ام تي ان داتا وانشط بي سوداني
ثانكس يادنيا
علشان اشتريتها ولا شنو
ديل يسرقوا الرصيد بطريقه عجيبه بعد الغلا دي كلها مامكفيهم يسرقوا الرصيد ويعملوا ليك كمان خدمات اجباري منهم يسرقوا رصيدك
الحمد لله نشطنا الليله
مجاني بس
واي لا عليك الله
من شبكتم القويه يعني ولا زايدنو لي شنو
نسال الل

In [35]:
from nltk.corpus import stopwords
from nltk import word_tokenize

arabicStopWords= stopwords.words("arabic")

for i in data['clean']:
    tokenizedRow = word_tokenize(i)
    commentWithNoStopWords= ' '.join([i for i in tokenizedRow if i not in arabicStopWords])
                

    data['clean'][i]=commentWithNoStopWords

In [36]:
for i in data['clean']:
    print(i)

نفسي يوم تكتبو السعر بدون مانسال
 لو قللتو الرسائل دي واديتونا ليها ميقات يكون اجمل
 واو
شكرا التوضيح مفيد اكرر الشكر سوداني الابداع والتميز
سوداني جميل
خدماتكم كلها ضابطه بس انا زعلانه من حاجه وحدة بس ( لي شلتو مني شارة أبرز المعجبين )
طبعا خجلانين تكتبوا السعر بعد الزيادات الأخيرة صح
الشبكه تعبانه شديد حاولوا حلو المشكله عشان م نشترك واشتراكنا يروح علي الفاضي ساكت تحياتي ليكم من منطقة ابو فسيسية حلة الدرحمان ابو طباعة الفجغ الجماعة
 بتبالق لكن نشطة قيقا و نزلت ملف و م ديرها تنتهي كمان
الشبكة عندنا كعبه جدا نتمني عن يتحسن ان شاءالله
ما عندنا شبكة لينا شهر وزيادة
سوداني تعبانة والله حتي ولو الخدمه مجان
صلحوا الشبكة اول بعد فالحين في الدفع بس
يوم العيد عشمتموني قلتو لي منوزع هدايا باقات نت مجانية عيدية وما اديتني حاجة منها
والله اول مره يكون كلامكم مشيت اشتريت شريحه جديده وفعلا لقيت فيها غيغا ونص هديه
لسوء الخدمة وقفت الحساب لاني بدفع وما مستفيد من خدمة الخط اتفاجئ بإن الشريحة بقيت غير خط لماذا؟
اناسوداني هيي هيي انااصل الحضارة
وقفو اي باقه وصلحو الشبكه
نستمتع كيف وانتو رفعتوها السماء"


ياخ الواحد لو داير اعمل خدمه في الشهر دا مافي قروش من المفطر تكون عملين شي لبلدكم السودان دا ما تزيدو ليهم الاسعار
خفضو تعرفت الاتصالات والا نترنيت بس دا ء اكبر عمل انساني
موفقين
خفضوا لينا الاسعار عشان اسي الناس بقت تهرب من سوداني لشركات تانيه شرحيتي اقفلواها ختيها في شباك
دام عزك يا وطن سوداني
معك في كل الظروف
متعكم الله بالصحه والعافيه يارب العالمين
ربنا يوفقكم
من عاني مشكله في شبكه سوداني
نتمني لكم التوفيق والنجاح
كضابين
احلي شبكه تواصل
صلحو شبكم هنا اول
عالم جميل
والله زول غيركم قتل الانسانيه الفينا مافي
بالله عليكم اعملوا فينا خير وما تسحبوا رصيدنا عشان ما ندعي عليكم دعوه مظلوم
سوداني لم تكن شبكهعندنااطلاق لانهاشبكه خاصهلجمع امولنابدون اي خدمه لااتصال ولانت ولاحتي رساءل بس ابراج وخطوط اشعارهفي الهواتف
اعملو خير فينا نحنا اور نزلو لينا سعر التنشيط ده قال انسانيه انتو لو عندكم انسانيه م كان وصلتو سعر التنشيط لي الدرجه دي
العمل التطوعي ده يشمل النصب وتزيد الباقات
الشباك الوحيد المدخل قروش لاكيزان هي شركه سوداني
مع انو الشركه شغاله في ست دول خارجيه جمعه مباركه و دوما سباقه بالمبادارت

سوداني واصيل في دمو جاري النيل
نجم عالي حلمو غالي ونور النجوم دليل
قدامو سكه نجاح وبيعرف كيف يبدا صاح
كل الحب
باقات الداتا زدنا خياراتا
حلو الكلام
احلا ناس
قدام
موفقين
بالتوفيق
كل الجمال كل الحب لك سوداني
انشاء الله تكوني في نصيبي يا رب العالمين
بالتوفيق ان شاءالله يارب
سوداني افضل اختيار
كل الحب والاحترام والتقدير
شركه محترمه
ابداع والله
بالتوفيق شركه سوداني
سوداني
اقوي شبكه في السودان
وافضل خدمات
ورقي مع المشتركين موفقين داءما
سوداني و خليك سوداني
سوداني خيارك الاول و الافضل
جميل
والله كلمه زدنا دي بقت تجيب لي وجع بطن ما ممكن تختوها في بدايه الجمله كدا
الحمد لله كثيرا
سوداني
شكرا علي هذا
حلو شديد
اوع اكون الزياده في الخيارات وفي الاسعار كمان
سوداني
موفقين
سوداني الحب
اللهم وفقنا
ما عندو سوداني اشتري سوداني
شكرا سوداني
احلي شركه
دي بتعات شنو
سوداني عالم راءع
سوداني خيارنا الاول
غيروها بي زادانا
يعلمو ليك ضجه
خليك سوداني
شغل ممتاز
كل الجمال
ان شاءالله
موفقين يارب
سوداني الخير
ياخ والله م فيها كلام لكن بالجد كداا سخنتوناا عديل كداا عليكم الله خفضو الاسعار دي
بالتوفيق
بالتوفيق
تماام
اعجب

In [38]:
from aranorm import normalize_arabic_text
for i in data['clean']:

    data['clean'][i]=normalize_arabic_text(i)


In [39]:
for i in data['clean']:
    print(i)

نفسي يوم تكتبو السعر بدون مانسال
 لو قللتو الرسائل دي واديتونا ليها ميقات يكون اجمل
 واو
شكرا التوضيح مفيد اكرر الشكر سوداني الابداع والتميز
سوداني جميل
خدماتكم كلها ضابطه بس انا زعلانه من حاجه وحدة بس ( لي شلتو مني شارة أبرز المعجبين )
طبعا خجلانين تكتبوا السعر بعد الزيادات الأخيرة صح
الشبكه تعبانه شديد حاولوا حلو المشكله عشان م نشترك واشتراكنا يروح علي الفاضي ساكت تحياتي ليكم من منطقة ابو فسيسية حلة الدرحمان ابو طباعة الفجغ الجماعة
 بتبالق لكن نشطة قيقا و نزلت ملف و م ديرها تنتهي كمان
الشبكة عندنا كعبه جدا نتمني عن يتحسن ان شاءالله
ما عندنا شبكة لينا شهر وزيادة
سوداني تعبانة والله حتي ولو الخدمه مجان
صلحوا الشبكة اول بعد فالحين في الدفع بس
يوم العيد عشمتموني قلتو لي منوزع هدايا باقات نت مجانية عيدية وما اديتني حاجة منها
والله اول مره يكون كلامكم مشيت اشتريت شريحه جديده وفعلا لقيت فيها غيغا ونص هديه
لسوء الخدمة وقفت الحساب لاني بدفع وما مستفيد من خدمة الخط اتفاجئ بإن الشريحة بقيت غير خط لماذا؟
اناسوداني هيي هيي انااصل الحضارة
وقفو اي باقه وصلحو الشبكه
نستمتع كيف وانتو رفعتوها السماء"


موفقين دوما ان شاء الله
موفقين باذن الله
ربنا يوفيقكم
ماشاء الله تبارك الله
هوي والله انتو بتبالغو والله م ممكن القيقا بقت بي حرام والله نحنا ناس كادحين غلابه يعني م ننشط ولا شنو زمان كنت بنشط قيقا كل يوم هسي من اسبوع لي اسبوع عشان اقدر الم حنو علينا يخ
موفقين ان شاء الله
بالتوفيق والنجاح
داءما موفقين يارب
روعه والله
اسعد الله مساءكم بالخير والبركات
الف شكر
حنينه
بالتوفيق ان شاء الله
اعملوا لينا باقات هديه ي جاحدين
اب
من نجاح لي نجاح
عرض ممتاز
شكراسوداني ومافي كلام
خليك سوداني
قلبي اباكم بقيتو غالييين وشيكتكم زفت تعالو شيلو شريحتكم دي ساي
انتو بتعزمو في الناس عشان ينوروكم طيب ماتعزوا الناس العندكم اول
تسلم سوداني
لكمالتقدم والازدهارسوداني
خليك سوداني
بالتوفيق
سوداني عالم جميل
التوفيق والنجاح
خلاص ظابط اتفقنا استمرررروا
جميل
سوداني
سوداني في اي مكان
قيقا و نص في الشهر دي يكملها في يوم اها و باقي الشهر
الشهر الرابع اجدعها طيب
الله يوفقك
معرض راءعه
كضب كضب
بالتوفيق ان شاء اللهيا رب اكتب لنا النصيب الجميل و من القدر اجمله
شبكتنا كعبه عديل كدا تعالوا اول حاجه ظبطها كويس وتاني حاجه نحنا مستع

In [53]:
# import module
import openpyxl

# load excel with its path
wrkbk = openpyxl.load_workbook("All_data.xlsx")

sh = wrkbk.active

comments=[]
# iterate through excel and display data
for i in range(1, sh.max_row+1):
    comment= dict()
    comment['comment'] = sh.cell(row=i, column=1).value
    comment['label'] =sh.cell(row=i, column=2).value
    print(comment)
   
    comments.append(comment)
    

{'comment': 'comment', 'label': 'label'}
{'comment': 'نفسي يوم تكتبو السعر بدون مانسال', 'label': 'سلبي'}
{'comment': 'طيب ما تشرحو طريقه الاشتراك في الباقه دي', 'label': 'محايد'}
{'comment': ' لو قللتو الرسائل دي واديتونا ليها ميقات يكون اجمل', 'label': 'سلبي'}
{'comment': 'رمز الاشتراك شنو', 'label': 'محايد'}
{'comment': ' واو', 'label': 'ايجابي'}
{'comment': ' الاشتراك كيف', 'label': 'محايد'}
{'comment': 'دي خط بتمشي المركز عشان تتنشط ليك', 'label': 'محايد'}
{'comment': 'حاولو اعملو كود للخدمات دي مش الزول كل مره يمشي يحول شريحته لي خدمه', 'label': 'محايد'}
{'comment': 'شكرا التوضيح مفيد اكرر الشكر سوداني الابداع والتميز', 'label': 'ايجابي'}
{'comment': 'السعر ٣٣٤٠', 'label': 'محايد'}
{'comment': 'تم', 'label': 'محايد'}
{'comment': 'نشطت غيغا قبل شويه ونزلت ملف 1GB والباقه خلصت عايزكم تفيدوني ليه خلصت الباقه وشكرا', 'label': 'محايد'}
{'comment': ' عشان انت ساريا', 'label': 'محايد'}
{'comment': 'سوداني جميل', 'label': 'ايجابي'}
{'comment': 'كم سعر ١٠ قيقا شريحه خط', 'label': 'محايد'}

{'comment': 'غليتو النت ماقلنا حاجه لكن ماتصلحو النت حقكم دا بنقاطعكم طوالي يا وش الفقر', 'label': 'سلبي'}
{'comment': 'اقسم بالله حرام عليكم ياخ كل مره زياده ان شاء الله م تنفعكم', 'label': 'سلبي'}
{'comment': 'تبا لكم حتي الدولارات الجايه من برا عاوزين تلموها عليكم', 'label': 'سلبي'}
{'comment': 'خلاص السوق الاسود وقف عايزين حق المساكين ديل', 'label': 'سلبي'}
{'comment': 'اخيرا الناس بقت تتجهه في لحلول ناجحه', 'label': 'ايجابي'}
{'comment': 'ههههه ان شاءالله عليكم بالساحق والماحق', 'label': 'سلبي'}
{'comment': 'بااااااااالغتو ف الزياده ياخ طيب كان حسبي الله ونععم الوكيل وشبكه خررررررررررا بس', 'label': 'سلبي'}
{'comment': 'ازاله التمكين ورونا مقرها وين عشان نبلغ فيهم ناس سوداني', 'label': 'سلبي'}
{'comment': 'اعوذ بالله منكم خافو الله ماف اي مبرر تزيدو سعر النت', 'label': 'سلبي'}
{'comment': 'اسوء دوله و خدمه والانترنت وكل شيء نحنا نكره السودان نحنا ما عندنا وطنيه للدوله', 'label': 'سلبي'}
{'comment': 'والله العظيم قروشي يا سوداني ماعافيها لي م حقت النت', 'label': 'سلبي'}
{'comment':

{'comment': 'والله ينتقم منكم كلكم ي حراميه ي اشرار', 'label': 'سلبي'}
{'comment': 'بكره بطل الشراءح زاتووو كرهتوووونا الانتر وامس الانتر ضاربنااا بي شاكوش الواحد يملي التلفون ويلعب بس لا نيت لا بوبجي', 'label': 'سلبي'}
{'comment': 'عايزين نعمل شركه مساهمه عامه', 'label': 'سلبي'}
{'comment': 'الاسم شركه الشعب للاتصالات', 'label': 'ايجابي'}
{'comment': 'بطرح اسهم لكل الشعب', 'label': 'محايد'}
{'comment': 'اساسا الشبكه زفت زياده الاسعار لزوم شنو', 'label': 'سلبي'}
{'comment': 'علي العموم نحنا اساسا تبع المجاني بس', 'label': 'سلبي'}
{'comment': 'ازيجوهم مويه نار ان شاء الله النت دي الحاجه المصبرنا علي البلد دي', 'label': 'سلبي'}
{'comment': 'اريبا وخلاص', 'label': 'سلبي'}
{'comment': 'ياخ الله يلعنكم ويلعن شبكتكم وشركتكم يامال السحت', 'label': 'سلبي'}
{'comment': 'هي اصلا من امس شبكه سوداني ماف', 'label': 'سلبي'}
{'comment': 'ح نخليكم', 'label': 'سلبي'}
{'comment': 'بلد ماشه عكس', 'label': 'سلبي'}
{'comment': 'انتو شركات الاتصال ماعليها ضراءب ولا شنو', 'label': 'سلبي'}
{'comment': 'مخرجات

In [54]:
df = pd.DataFrame(comments, columns =['label','comment'])


In [57]:
df

,label,comment
0,label,comment
1,سلبي,نفسي يوم تكتبو السعر بدون مانسال
2,محايد,طيب ما تشرحو طريقه الاشتراك في الباقه دي
3,سلبي,لو قللتو الرسائل دي واديتونا ليها ميقات يكون ...
4,محايد,رمز الاشتراك شنو
...,...,...
3069,ايجابي,خليك سوداني
3070,ايجابي,سوداني
3071,ايجابي,سوداني الاقوي والافضل
3072,ايجابي,خليك سوداني
